In [3]:
# Imports
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, datasets
from torch.utils.data import DataLoader, random_split
import os

FER_213_PATH = "/kaggle/input/fer2013/train"

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [5]:
# Getting data ready
tranform = transforms.Compose([
    transforms.Resize((48, 48)),
    transforms.Grayscale(num_output_channels=3),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(20),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],  # ImageNet stats
                         std=[0.229, 0.224, 0.225])
])

In [6]:
# Importing dataset
dataset = datasets.ImageFolder(root=FER_213_PATH, transform=tranform)

train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

print("Classification Options:", dataset.class_to_idx)
print("Number of training samples:", len(train_dataset))
print("Number of validation samples:", len(val_dataset))

Classification Options: {'angry': 0, 'disgust': 1, 'fear': 2, 'happy': 3, 'neutral': 4, 'sad': 5, 'surprise': 6}
Number of training samples: 22967
Number of validation samples: 5742


In [8]:
# Load a pretrained RestNet18 model
from torchvision.models import resnet18, ResNet18_Weights
from torchsummary import summary

weights = ResNet18_Weights.DEFAULT
model = resnet18(weights=weights)

# Modify the final layer for 7 classes
model.fc = nn.Linear(model.fc.in_features, 7)

# Transfer learning only on the last residual block and the final fully connected
for name, param, in model.named_parameters():
    if "layer4" not in name and "fc" not in name:
        param.requires_grad = False
    else:
        param.requires_grad = True

model = model.to(device)

summary(model, input_size=(3, 224, 224))

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 185MB/s]


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]          36,864
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
            Conv2d-8           [-1, 64, 56, 56]          36,864
       BatchNorm2d-9           [-1, 64, 56, 56]             128
             ReLU-10           [-1, 64, 56, 56]               0
       BasicBlock-11           [-1, 64, 56, 56]               0
           Conv2d-12           [-1, 64, 56, 56]          36,864
      BatchNorm2d-13           [-1, 64, 56, 56]             128
             ReLU-14           [-1, 64,

In [9]:
loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-4)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=3)

In [11]:
# Training loop
num_epochs = 30
best_val_accuracy = 0.0
patience = 5
early_stop_counter = 0

for epoch in range(num_epochs):
    model.train()
    total_loss = 0.0
    correct = 0
    total = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        outputs = model(images)
        loss = loss_function(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

    train_acc = 100 * correct / total

    # Validation data
    model.eval()
    val_correct = 0
    val_total = 0

    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            val_correct += (predicted == labels).sum().item()
            val_total += labels.size(0)

    val_acc = 100 * val_correct / val_total
    scheduler.step(val_acc)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss/len(train_loader):.4f}, "
          f"Train Accuracy: {train_acc:.2f}%, Val Accuracy: {val_acc:.2f}%")
    
    # Early stopping
    if val_acc > best_val_accuracy:
        best_val_accuracy = val_acc
        early_stop_counter = 0
        torch.save(model.state_dict(), 'best_FER_2013_model.pth')
    else:
        early_stop_counter += 1
        if early_stop_counter >= patience:
            print("Early stopping triggered, training stopped.")
            break

# Some verbose output
print(f"Best validation accuracy: {best_val_accuracy:.2f}%")
print("Training complete. Model saved as 'best_FER_2013_model.pth'.")

Epoch [1/30], Loss: 1.5317, Train Accuracy: 40.84%, Val Accuracy: 41.55%
Epoch [2/30], Loss: 1.4667, Train Accuracy: 43.35%, Val Accuracy: 42.96%
Epoch [3/30], Loss: 1.4273, Train Accuracy: 44.90%, Val Accuracy: 45.89%
Epoch [4/30], Loss: 1.3859, Train Accuracy: 47.13%, Val Accuracy: 44.95%
Epoch [5/30], Loss: 1.3663, Train Accuracy: 47.96%, Val Accuracy: 44.95%
Epoch [6/30], Loss: 1.3374, Train Accuracy: 49.20%, Val Accuracy: 46.45%
Epoch [7/30], Loss: 1.3163, Train Accuracy: 49.63%, Val Accuracy: 46.59%
Epoch [8/30], Loss: 1.2909, Train Accuracy: 50.99%, Val Accuracy: 46.45%
Epoch [9/30], Loss: 1.2681, Train Accuracy: 51.89%, Val Accuracy: 47.58%
Epoch [10/30], Loss: 1.2465, Train Accuracy: 52.45%, Val Accuracy: 47.37%
Epoch [11/30], Loss: 1.2253, Train Accuracy: 53.65%, Val Accuracy: 47.47%
Epoch [12/30], Loss: 1.2045, Train Accuracy: 54.59%, Val Accuracy: 48.64%
Epoch [13/30], Loss: 1.1864, Train Accuracy: 55.06%, Val Accuracy: 49.20%
Epoch [14/30], Loss: 1.1588, Train Accuracy: 56